In [1]:
%cd ../

/home/hoanghu/projects/Food-Waste-Optimization


In [11]:
from pathlib import Path
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from darts.models import ARIMA, LinearRegressionModel
from darts import TimeSeries
from statsmodels.tsa.stattools import adfuller, kpss

In [3]:
plt.style.use('seaborn-v0_8')
plt.rcParams.update({'font.size': 8})

In [4]:
path_root_trained_model = Path("trained_models")
path_processed = Path("experiments_hoangle/processed/fact.csv")

# Read dataset

In [7]:
df = pd.read_csv(path_processed, header=0, parse_dates=[0])

df.head(5)

,date,restaurant,num_fish,num_chicken,num_vegetable,num_meat,num_NotMapped,num_vegan,num_customer_in,num_customer_out,num_rcpts,amnt_waste_customer,amnt_waste_coffee,amnt_waste_kitchen,amnt_waste_hall
0,2023-01-02,Chemicum,85.0,0.0,0.0,171.0,1.0,91.0,NaN,NaN,272.0,4.70,1.2,12.0,0.0
1,2023-01-03,Chemicum,163.0,0.0,32.0,78.0,1.0,120.0,NaN,NaN,327.0,5.00,1.4,14.8,0.0
2,2023-01-04,Chemicum,70.0,0.0,0.0,218.0,3.0,137.0,NaN,NaN,351.0,4.15,4.0,7.1,0.0
3,2023-01-05,Chemicum,232.0,85.0,0.0,2.0,4.0,178.0,NaN,NaN,437.0,10.00,3.3,8.5,0.0
4,2023-01-06,Chemicum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Train & save models

In [23]:
RESTAURANTS = df['restaurant'].unique()
WASTE_TYPES = ['amnt_waste_customer', 'amnt_waste_coffee', 'amnt_waste_kitchen', 'amnt_waste_hall']

# This cutoff date is used for all 3 forecasting models of 3 corresponding restaurants
# to ensure the datetime of predicted series among 3 restaurants are the same
CUTOFF_DATE = '2024-05-08'  

In [24]:
for restaurant in RESTAURANTS:
    df_restaurant_raw = df[df['restaurant'] == restaurant][['date', 'restaurant', *WASTE_TYPES]]\
        .drop(columns='restaurant')
    df_restaurant_raw = df_restaurant_raw[~df_restaurant_raw.isna().any(axis=1)]

    # Create dataframe full dates with data from 'df_restaurant_raw'
    freq = pd.offsets.BusinessDay()

    date_start = df_restaurant_raw['date'].min()

    date_range = pd.date_range(start=date_start, end=CUTOFF_DATE, freq=freq)

    df_restaurant = pd.DataFrame({'date': date_range})
    df_restaurant = df_restaurant.merge(df_restaurant_raw, on='date', how='left')

    # Fill nan values by column-wise mean
    fillna_vals = df_restaurant.mean(axis=0)

    for waste_type in WASTE_TYPES:
        df_restaurant.fillna({waste_type: fillna_vals[waste_type]}, inplace=True)

    # Create series
    series = TimeSeries.from_dataframe(
        df=df_restaurant,
        time_col='date',
        freq=freq,
        fill_missing_dates = True,
        value_cols=WASTE_TYPES
    )

    # Define model
    add_encoders = {
        'cyclic': {
            'past': ['dayofweek']
        },
        'datetime_attribute': {'past': ['dayofweek']},
    }
    model = LinearRegressionModel(lags=5, lags_past_covariates=5, add_encoders=add_encoders, output_chunk_length=5)

    # Train model
    model.fit(series)

    # Save model
    path_model = path_root_trained_model / "biowaste" / f"{restaurant}.pt"
    path_model.parent.mkdir(exist_ok=True, parents=True)

    model.save(path_model)

# Load models

In [25]:
add_encoders = {
        'cyclic': {
            'past': ['dayofweek']
        },
        'datetime_attribute': {'past': ['dayofweek']},
    }

models = {'biowaste': {}}

for restaurant in RESTAURANTS:
    path_model = Path(f"trained_models/biowaste/{restaurant}.pt")

    models['biowaste'][restaurant] = LinearRegressionModel(
        lags=5,
        lags_past_covariates=5,
        add_encoders=add_encoders,
        output_chunk_length=5
    ).load(path_model)

In [26]:
num_of_days = 3

predictions = {}

# Forecast the future
for restaurant in RESTAURANTS:
    pred = models['biowaste'][restaurant].predict(num_of_days)

    df_pred = pred.pd_dataframe().reset_index()
    df_pred['date'] = df_pred['date'].dt.strftime(r"%Y-%m-%d")

    for row in df_pred.itertuples():
        if row.date not in predictions:
            predictions[row.date] = {'date': row.date}

        predictions[row.date][restaurant] = {
            'amnt_waste_customer': row.amnt_waste_customer,
            'amnt_waste_coffee': row.amnt_waste_coffee,
            'amnt_waste_kitchen': row.amnt_waste_kitchen,
            'amnt_waste_hall': row.amnt_waste_hall,
        }

print(json.dumps(predictions, indent=2))

{
  "2024-05-09": {
    "date": "2024-05-09",
    "Chemicum": {
      "amnt_waste_customer": 15.335807096971186,
      "amnt_waste_coffee": 6.749373675160983,
      "amnt_waste_kitchen": 13.120679698841172,
      "amnt_waste_hall": 12.76089686219699
    },
    "Physicum": {
      "amnt_waste_customer": -0.09098052978515625,
      "amnt_waste_coffee": 11.28973388671875,
      "amnt_waste_kitchen": 12.328250885009766,
      "amnt_waste_hall": 0.17769622802734375
    },
    "Exactum": {
      "amnt_waste_customer": 5.2109375,
      "amnt_waste_coffee": 1.4091796875,
      "amnt_waste_kitchen": 8.2734375,
      "amnt_waste_hall": -1.39208984375
    }
  },
  "2024-05-10": {
    "date": "2024-05-10",
    "Chemicum": {
      "amnt_waste_customer": 14.879137084963158,
      "amnt_waste_coffee": 8.28143831276184,
      "amnt_waste_kitchen": 16.26109595145685,
      "amnt_waste_hall": 14.719872271067954
    },
    "Physicum": {
      "amnt_waste_customer": -0.07980728149414062,
      "amnt_waste